Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [52]:
import numpy as np
import pandas as pd

import re
import nltk
nltk.download('stopwords')

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB

from nltk.corpus import stopwords
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score

from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [53]:
comments = pd.read_csv('/datasets/toxic_comments.csv')

In [54]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


В датасете 159 571 записей

    text - комментарии пользователей, тип object
    toxic - признак токсичности комментария:
        0 - комментарий положительный
        1 - комментарий негативный

In [55]:
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [56]:
#пропуски
comments.isna().mean()

text     0.0
toxic    0.0
dtype: float64

Пропуски отсутствуют

In [57]:
#дубликаты
comments.duplicated().sum()

0

Дубликаты отсутствуют

In [58]:
comments['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

### Выводы

    В датасете 159 571 записей
    Два признака - комментарии пользователей и классификатор тональности комментария
    Данные несбалансированые: негативные комментарии составляют 10,17% от всей выборки
    Дубликатов и пропусков в датасете нет

### Очистка

In [59]:
#Отчистим текст от лишних знаков, оставим только буквы и пробелы. Приведем текст к нижнему регистру. Проведем лемматизацию
corpus = comments['text'].values
lemmas = []
stemmer = WordNetLemmatizer()

for sen in range(0, len(corpus)):
    # Удаляем все специальные символы
    document = re.sub(r'\W', ' ', str(corpus[sen]))
    
    # Удаляем все единичные символы
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Удаляем единичные символы из начала строки
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Заменяем несколько пробелов на один
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Удаляем префикс 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Переводим заглавные буквы в строчные, что облегчит дальнейшую лемматизацию и обучение модели
    document = document.lower()
    
    # Проведем лемматизацию 
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    lemmas.append(document)    

In [60]:
lemmas[:3]

['explanation why the edits made under my username hardcore metallica fan were reverted they weren vandalism just closure on some gas after voted at new york doll fac and please don remove the template from the talk page since m retired now 89 205 38 27',
 'd aww he match this background colour m seemingly stuck with thanks talk 21 51 january 11 2016 utc',
 'hey man m really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info']

In [61]:
#Переведем леммамы в Series и присоединим к датасету
lemmas_series = pd.Series(lemmas, name = 'lemmas')
comments_new = pd.concat([comments, lemmas_series], axis = 1)
comments_new.head()

,text,toxic,lemmas
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour m seemin...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man m really not trying to edit war it jus...
3,"""\nMore\nI can't make any real suggestions on ...",0,more can make any real suggestion on improveme...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


### Векторизация

In [62]:
#выделим целевой признак в отдельный датасет
comments_features = comments_new.drop(['text','toxic'], axis=1)
comments_target = comments_new['toxic']

In [63]:
#разобьем данные на выборки
X_train, X_rest, y_train, y_rest = train_test_split(comments_features, comments_target, test_size=0.3, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=12345)

In [64]:
#проверим размер выборок
display(X_train.shape, X_valid.shape, X_test.shape)

(111699, 1)

(23936, 1)

(23936, 1)

In [65]:
#Создадим счётчик, укажем стоп-слова
stopwords = set(nltk_stopwords.words('english'))

In [66]:
#Создадим corpus комментариев обучающей и тестовой выборки
corpus_train = X_train['lemmas'].values
corpus_valid = X_valid['lemmas'].values
corpus_test = X_test['lemmas'].values

In [67]:
#Обучим конвертер на выборке corpus_train
tf_idfconverter = TfidfVectorizer(max_features=6000, min_df=5, max_df=0.7, stop_words=stopwords)
tf_idfconverter.fit(corpus_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.7, max_features=6000,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [68]:
#Конвертируем все выборки в матрицы
X_train_tf = tf_idfconverter.transform(corpus_train)
X_valid_tf = tf_idfconverter.transform(corpus_valid)
X_test_tf = tf_idfconverter.transform(corpus_test)

In [69]:
#проверим размерность выборок
display(X_train_tf.shape, X_valid_tf.shape, X_test_tf.shape)

(111699, 6000)

(23936, 6000)

(23936, 6000)

# 2. Обучение

In [70]:
res=[]

In [71]:
#функция обучения моделей
def ml_models(models, ft, tt, fv, tv):
    model = models
    model.fit(ft, tt)
    
    predictions_valid = model.predict(fv)
    print('f1 = {:.2f}'.format(f1_score(tv, predictions_valid)))
    return f1_score(tv, predictions_valid)

#### LogisticRegression

In [72]:
res.append(ml_models(LogisticRegression(random_state=12345), X_train_tf, y_train, X_valid_tf, y_valid))

f1 = 0.75


#### LinearSVC

In [73]:
res.append(ml_models(LinearSVC(random_state=12345), X_train_tf, y_train, X_valid_tf, y_valid))

f1 = 0.77


#### SGDClassifier

In [74]:
res.append(ml_models(SGDClassifier(random_state=12345), X_train_tf, y_train, X_valid_tf, y_valid))

f1 = 0.67


#### MultinomialNB

In [76]:
res.append(ml_models(MultinomialNB(), X_train_tf, y_train, X_valid_tf, y_valid))

f1 = 0.68


In [77]:
#сводная таблица моделей
result = pd.DataFrame(data=res, columns=['f1']).rename(index={0: 'LogisticRegression', 1: 'LinearSVC', 2: 'SGD', 3: 'MultinomialNB'})
display(result)

,f1
LogisticRegression,0.749635
LinearSVC,0.774828
SGD,0.668277
MultinomialNB,0.680996


лучший показатель метрики у модели LinearSVC = 0.774828, а худший у MultinomialNB = 0.680996

In [78]:
res = []

In [79]:
#проведем подбор параметров для моделей
def gridsearch_result(model, parameters, f, t, scorer):
    gridsearch = GridSearchCV(estimator=model, param_grid = parameters, refit=True, scoring=scorer, cv=2)
    gridsearch = gridsearch.fit(f, t)
        
    print('Лучшие гиперпараметры: ', gridsearch.best_params_)
    return gridsearch.best_estimator_

#### LogisticRegression (GridSearchCV)

In [80]:
parameters = {
        'random_state' : [12345],
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 200]
        }
best_LogisticRegression = gridsearch_result(LogisticRegression(), parameters, X_train_tf, y_train, 'f1')

Лучшие гиперпараметры:  {'C': 1, 'penalty': 'l1', 'random_state': 12345}


In [81]:
res.append([ml_models(best_LogisticRegression, X_train_tf, y_train, X_valid_tf, y_valid), best_LogisticRegression.get_params()])

f1 = 0.78


#### LinearSVC (GridSearchCV)

In [82]:
parameters = {
        'max_iter' : range(1000, 2000, 100),
        'C' : [0.1, 1, 10, 100],
        'random_state' : [12345]        
        }
best_LinearSVC = gridsearch_result(LinearSVC(), parameters, X_train_tf, y_train, 'f1')

Лучшие гиперпараметры:  {'C': 1, 'max_iter': 1000, 'random_state': 12345}


In [83]:
res.append([ml_models(best_LinearSVC, X_train_tf, y_train, X_valid_tf, y_valid), best_LinearSVC.get_params()])

f1 = 0.77


#### SGDClassifier (GridSearchCV)

In [84]:
parameters = {
        'alpha' : [1, 1e-01, 1e-02, 1e-03, 1e-04, 1e-05, 1e-06],
        'penalty' : ['l1','l2'],
        'random_state' : [12345]        
        }
best_SGD = gridsearch_result(SGDClassifier(), parameters, X_train_tf, y_train, 'f1')

Лучшие гиперпараметры:  {'alpha': 1e-05, 'penalty': 'l1', 'random_state': 12345}


In [85]:
res.append([ml_models(best_SGD, X_train_tf, y_train, X_valid_tf, y_valid), best_SGD.get_params()])

f1 = 0.78


#### MultinomialNB (GridSearchCV)

In [86]:
parameters = {
        'alpha' : [1e-06, 1e-05, 1e-04, 1e-03, 1e-02, 1e-01, 1],
        }
best_MNB = gridsearch_result(MultinomialNB(class_prior=None, fit_prior=True), parameters, X_train_tf, y_train, 'f1')

Лучшие гиперпараметры:  {'alpha': 0.1}


In [87]:
res.append([ml_models(best_MNB, X_train_tf, y_train, X_valid_tf, y_valid), best_MNB.get_params()])

f1 = 0.69


In [88]:
#сводная таблица моделей с подбором параметров
result_gs = pd.DataFrame(data=res, columns=['f1 gs', 'gs params']).rename(index={0: 'LogisticRegression', 1: 'LinearSVC', 2: 'SGD', 3: 'MultinomialNB'})
display(result_gs)

,f1 gs,gs params
LogisticRegression,0.777804,"{'C': 1, 'class_weight': None, 'dual': False, ..."
LinearSVC,0.774828,"{'C': 1, 'class_weight': None, 'dual': True, '..."
SGD,0.778009,"{'alpha': 1e-05, 'average': False, 'class_weig..."
MultinomialNB,0.693888,"{'alpha': 0.1, 'class_prior': None, 'fit_prior..."


Лучший показатель исследуемой метрики у модели SGD = 0.778009, немного отстает LogisticRegression = 0.777804, а худший показатель у MultinomialNB = 0.693888

### Проверка на тестовой выборке

In [89]:
res = []

In [90]:
res.append(ml_models(best_LogisticRegression, X_train_tf, y_train,  X_test_tf, y_test))

f1 = 0.77


In [91]:
res.append(ml_models(best_LinearSVC, X_train_tf, y_train,  X_test_tf, y_test))

f1 = 0.77


In [92]:
res.append(ml_models(best_SGD, X_train_tf, y_train,  X_test_tf, y_test))

f1 = 0.76


In [93]:
res.append(ml_models(best_MNB, X_train_tf, y_train,  X_test_tf, y_test))

f1 = 0.68


In [94]:
#сводная таблица по тестовой выборке
result_test = pd.DataFrame(data=res, columns=['f1 test']).rename(index={0: 'LogisticRegression', 1: 'LinearSVC', 2: 'SGD', 3: 'MultinomialNB'})
display(result_test)

,f1 test
LogisticRegression,0.767206
LinearSVC,0.765764
SGD,0.763085
MultinomialNB,0.679671


3 из 4 моделей достигли значения метрики качества F1 не меньше 0.75: LogisticRegression, LinearSVC, SGD

лучшую метрику на тестовой выборке показала модель LogisticRegression = 0.767206, а худшая у модели MultinomialNB = 0.679671

In [95]:
#итоговая таблица по всем сводным таблицам
result_final = pd.concat([result, result_gs, result_test], axis="columns")

In [97]:
result_final

,f1,f1 gs,gs params,f1 test
LogisticRegression,0.749635,0.777804,"{'C': 1, 'class_weight': None, 'dual': False, ...",0.767206
LinearSVC,0.774828,0.774828,"{'C': 1, 'class_weight': None, 'dual': True, '...",0.765764
SGD,0.668277,0.778009,"{'alpha': 1e-05, 'average': False, 'class_weig...",0.763085
MultinomialNB,0.680996,0.693888,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.679671


# 3. Выводы

3 из 4 моделей достигли значения метрики качества F1 не меньше 0.75: LogisticRegression, LinearSVC, SGD

Лучший показатель f1 показывает модель LogisticRegression =  0.767206

Худший результат по показателю исследуемой метрики - у MultinomialNB = 0.679671

## BERT

In [98]:
#для ускорения работы возьмем для проверки только 1000 комментариев
batch_1000 = comments[:1000]

In [99]:
import transformers as ppb
#Загрузим предобученную модели DistilBERT и токенизатор
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [100]:
#Ограничим длину комментариев 512 символами
batch_1000['text'] = batch_1000['text'].str[:512]

In [101]:
#Проведем токенизацию данных
tokenized = batch_1000['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [102]:
#Приведем векторы к одному размеру путем прибавления к более коротким векторам идентификатора 0
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [103]:
#нули не несут значимой информации, отбросим эти токены и укажем нулевые и не нулевые значения
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape)

(1000, 230)


In [113]:
from tqdm import notebook
import torch
#Создадим эмбеддинги для комментариев пользователей, ограничив размер батча 20 объектами
batch_size = 20
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [114]:
#Запишем все эмбеддинги в матрицу признаков, выделим целевой признак и разделим данные на выборки
features = np.concatenate(embeddings)
target = batch_1000['toxic']

In [115]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.3, random_state=12345)

In [116]:
#Обучим модель LogisticRegression
ml_models(LogisticRegression(random_state=12345, class_weight = 'balanced'), features_train, target_train, features_test, target_test)

f1 = 0.79


0.7894736842105262

При выборке 1000 комментариев BERT совместно с LogisticRegression показал результат f1 = 0.7894736842105262, что тоже превышает значение 0.75